In [1]:
import tensorflow as tf
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
# import os
# import numpy as np
# keras = tf.keras

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data() # load dataset
train_images, test_images = train_images/255, test_images/255 # normalize

class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [3]:
import tensorflow_datasets as tfds
# make downloading progress bar dissable
tfds.disable_progress_bar()

# setattr(tfds.image_classification.cats_vs_dogs, '_URL',"https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")

# split data into 80% training, 10% testing, 10% validation
# not explaining this, see documentation
# seems to only be working on macos
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [4]:
# gonna resize to all be the same
IMG_SIZE = 160
def format_example(image,label):
    image = tf.cast(image, tf.float32)
    # image = (image/127.5) - 1
    # takes only color values above 127 in the original data, what the fuck is this???
    # i refuse to do this
    image = image/255.0
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    return image, label

train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

In [5]:
# gonna use MobileNetV2 from google
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE,
                                               include_top = False,
                                               weights = 'imagenet')

In [6]:
base_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 160, 160, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 80, 80, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 80, 80, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 80, 80, 32)           0         ['bn_Conv1[

- outputs a shape (32, 5, 5, 1280) tensor that is a feature extraction from our original (1, 160, 160, 3) image

In [7]:
BATCH_SIZE = 32

train_batches = train.shuffle(1000).batch(BATCH_SIZE)
validation_batches = validation.shuffle(1000).batch(BATCH_SIZE)

In [9]:
for image, _ in train_batches.take(1):
    pass

feature_batch = base_model(image)
feature_batch.shape

TensorShape([32, 5, 5, 1280])

In [10]:
base_model.trainable = False
base_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 160, 160, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 80, 80, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 80, 80, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 80, 80, 32)           0         ['bn_Conv1[

- instead of flattening feature map of base layer we will use global pooling layer that will average the entire 5x5 area of each 2d feature map and return a single 128- element vector per filter

In [11]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

- next add prediction layer which is a single dense neuron.
- only 1 neuron because we only have 2 classes to predict for

In [12]:
prediction_layer = tf.keras.layers.Dense(1)

In [13]:
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_160 (Func  (None, 5, 5, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 2259265 (8.62 MB)
Trainable params: 1281 (5.00 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


### training the model
- picking a low learning rate
- learning rate: how much we can modify the weights and biases in one step

In [14]:
base_learning_rate = 0.0001
model.compile(
    optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate),
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

In [15]:
initial_epochs = 3
validation_steps = 20

loss0, accuracy0 = model.evaluate(validation_batches, steps = validation_steps)

20/20 [==============================] - 3s 79ms/step - loss: 0.7154 - accuracy: 0.5391


In [17]:
history = model.fit(train_batches, epochs = initial_epochs, validation_data=validation_batches)

Epoch 1/3
582/582 [==============================] - 56s 95ms/step - loss: 0.0492 - accuracy: 0.9818 - val_loss: 0.0613 - val_accuracy: 0.9738
Epoch 2/3
582/582 [==============================] - 54s 93ms/step - loss: 0.0466 - accuracy: 0.9838 - val_loss: 0.0584 - val_accuracy: 0.9781
Epoch 3/3
582/582 [==============================] - 55s 94ms/step - loss: 0.0443 - accuracy: 0.9845 - val_loss: 0.0600 - val_accuracy: 0.9807


In [18]:
acc = history.history['accuracy']
acc

[0.9818377494812012, 0.9837721586227417, 0.9844707250595093]

In [19]:
# saving and loading the model
model.save('dogs_vs_cats.h5')
new_model = tf.keras.models.load_model('dogs_vs_cats.h5')

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
